# BULKING NOTEBOOK 

In [83]:
import numpy as np
import pandas as pd
import sqlite3
import datetime

# Create db instance, connection and cursor to db instance
name = "bulking_table"
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [84]:
path = '/Users/geraldlittlejr/Documents/vs_files/bulking/bulking_file.txt'
df = pd.read_csv(path)
df

,Date,weight(lbs),workout_days,missed_meals,protein,creatine,Unnamed: 6
0,Nov202023,200.2,4.0,3,1,1,NaN
1,Nov272023,202.2,5.0,1,1,1,NaN
2,Dec042023,206.6,4.5,0,0,1,NaN
3,Dec112023,207.6,4.5,2,1,1,NaN
4,Dec182023,208.6,2.0,6,0,0,NaN
5,Dec262023,204.8,1.0,7,0,0,NaN
6,JAN022024,207.5,3.0,5,1,1,NaN
7,Jan092024,204.8,3.0,5,1,1,NaN
8,JAN162024,206.5,1.0,3(?),1,1,NaN
9,Jan222024,211.4,5.0,1,1,1,NaN


In [85]:
# CREATE TABLE FROM PANDAS
df.to_sql(name, con, if_exists='replace', index=False)

17

In [86]:
# CHECC THAT A TABLE WAS CREATED
res = cur.execute(f"SELECT name FROM sqlite_master")
res.fetchone()

('bulking_table',)

In [87]:
# CHECC FOR NON EXISTANT TABLE
res = cur.execute(f"SELECT name FROM sqlite_master WHERE name='foo'")
res.fetchone() is None

True

In [88]:
# Table Transformations
def table_transform(df):

    def clean_numerical(weight):
        import re
        # Remove any non-numeric characters using regular expressions
        return re.sub(r'[^\d.]+', '', weight)

    df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%b%d%Y')
    df['weight(lbs)'] = df['weight(lbs)'].astype('str').apply(clean_numerical)
    df['weight(lbs)'] = df['weight(lbs)'].astype('float').round(1)
    df['workout_days'] = df['workout_days'].astype('int')   
    df['missed_meals'] = df['missed_meals'].astype('str').apply(clean_numerical)
    df['missed_meals'] = df['missed_meals'].astype('int')
    df = df.drop(columns=['Unnamed: 6'], axis=1)

    return df

In [89]:
try:
    df_silver = table_transform(df)
    
    # WRITE SILVER TABLE TO DB FROM PANDAS
    silver = name + '_' + 'silver'
    df_silver.to_sql(silver, con, if_exists='replace', index=False)

    # CHECC THAT SILVER TABLE WAS CREATED
    res = cur.execute(f"SELECT name FROM sqlite_master")
    print(res.fetchall())

    # Create DataFrame from SQLite table
    silver_table = pd.read_sql_query(f"SELECT * FROM {silver}", con)
    
except AttributeError as e:
    print('There was an error with the transformation function:', e)
except sqlite3.OperationalError as e:
    print('There was an error with the SQLite operation:', e)
except Exception as e:
    print('An unexpected error occurred:', e)
finally:
    print(silver_table)

[('bulking_table',), ('bulking_table_silver',)]
                   Date  weight(lbs)  workout_days  missed_meals  protein  \
0   2023-11-20 00:00:00        200.2             4             3        1   
1   2023-11-27 00:00:00        202.2             5             1        1   
2   2023-12-04 00:00:00        206.6             4             0        0   
3   2023-12-11 00:00:00        207.6             4             2        1   
4   2023-12-18 00:00:00        208.6             2             6        0   
5   2023-12-26 00:00:00        204.8             1             7        0   
6   2024-01-02 00:00:00        207.5             3             5        1   
7   2024-01-09 00:00:00        204.8             3             5        1   
8   2024-01-16 00:00:00        206.5             1             3        1   
9   2024-01-22 00:00:00        211.4             5             1        1   
10  2024-01-29 00:00:00        210.4             1             3        0   
11                 None     